# Trigger calibration notebook

This notebook was made to calibrate and troubleshoot RP's trigger problems.

#### Preamble

Import and initialize everything needed

In [2]:
%load_ext autoreload
%autoreload 2

In [33]:
import matplotlib.pyplot as plt
import numpy as np
import time

In [59]:
from ucnpexp.instruments import Spectrometer
spec = Spectrometer.constructor_default()

#### Experiment 1

In this and the following experiments from now on unless otherwise specified, the hardware configuration is the following:

Instruments:
1. HP33120A Function Generator (FG)
    * Freq 100Hz
    * Amp 7Vpp
    * Offset 2.5VDC
    * Square wave 50% duty cycle (DC)
1. Tektronix TDS1002B Oscilloscope (OS)
1. Red Pitaya (RP)
1. Voltage divider (VD) on Proto board, R1 = 160, R2 = 2*160 

Connections are:
FG -> BNC T OS CH1 BNC T -> BNC T RP CH1 BNC T -> VD -> RP pin p0 (external trigger)

Voltage divider is used to feed RP external trigger GPIO P0 with 3.3 VPP Square wave instead of 5V.

With this configuration the trigger signal (RP GPIO P0) and the measured signal (RP CH1) are the same.
The goal is to figure out why the trigger sometimes triggers when there is no flank.
The hypothesis is that if you ask the RP for oscilloscope screens faster than the buffer was filled (in this case should be 0.1 seconds), then the returned data wasn't triggered at the right time.
To figure this we are going to ask for two triggered RP screens of 0.1 seconds with different delay times in the middle of them, save that data and then makea histogram of difference in trigger times for each delay time.
The key line of code here is the ```spec._osc.set_trigger(num)``` which sets what trigger is going to be used.
Supposedly, the number 6 and 7 correspond to external trigger with different slope triggering, infered by reading code in [here](https://github.com/search?q=repo%3ARedPitaya%2FRedPitaya%20switch(trig_source)&type=code).
The other option is that either number 5 or 2**5 correspond to the external trigger infered by reading the ADC streaming table [here](https://redpitaya.readthedocs.io/en/latest/developerGuide/software/build/fpga/regset/in_dev/stream_app.html).

In experiment 1 we use the number 5.

In [ ]:
spec._osc.set_measurement_time(0.1)
spec._osc.set_trigger(5, edge='pos', level=1.0)

delays = np.linspace(0.01, 1, 10)

for delay in delays:
    time.sleep(1)


In [60]:
spec._osc.set_measurement_time(0.1)
spec._osc.set_trigger(0x40000000 + 8, edge='pos', level=1.0)


In [61]:
screen = spec._osc.get_triggered()
plt.plot(screen)

TimeoutError: result expired